In [1]:
import psycopg2
import os

In [10]:

host = os.environ["PGHOST"]
port = os.environ["PGPORT"]
dbname = 'udacity'
user = os.environ["PGUSER"]
password = os.environ["PGPASSWORD"]

try:
    connection = psycopg2.connect(
         host = host
        ,port = port
        ,dbname = dbname
        ,user = user
        ,password = password
     )
    
    connection.set_session(autocommit=True)
    
except psycopg2.Error as error:
    print("Error: could not connect to Postgres")
    print(error)

try:
    cursor = connection.cursor()
    
except psycopg2.Error as error:
    print("Error: could not get cursor to the Database")
    print(error)

#### create the `sales` table

In [15]:

try:
    cursor.execute(
        """
            DROP TABLE IF EXISTS
                public.sales
            ;

            CREATE TABLE
                public.sales(
                     transaction_id SMALLINT
                    ,amount_spent FLOAT8
                )
        """
    )
    
except psycopg2.Error as error:
    print("Error: issue creating table")
    print(error)

#### insert data into the `sales` table

In [17]:

sales_data = [
     (1,40)
    ,(2,19)
    ,(3,45)
]

insert_into_sales_statement = """
    INSERT INTO
        public.sales(
             transaction_id
            ,amount_spent
        )
        
    VALUES(
         %s
        ,%s
    )
"""

try:
    cursor.executemany( 
        insert_into_sales_statement
       ,sales_data
    )
    
except psycopg2.Error as error:
    print("Error: could not insert data")
    print(error)

#### select statements are prepared
for table contents validation ahead:

In [18]:

transactions_3nf_select_statement = """
    SELECT
        *
        
    FROM
        public.transactions_3nf
"""

albums_sold_select_statement = """
    SELECT
        *
        
    FROM
        public.albums_sold
"""

employees_select_statement = """
    SELECT
        *
        
    FROM
        public.employees
"""

sales_select_statement = """
    SELECT
        *
        
    FROM
        public.sales
"""


In [24]:

table_names = [
     'transactions_3nf'
    ,'albums_sold'
    ,'employees'
    ,'sales'
]

select_statements_list = [
     transactions_3nf_select_statement
    ,albums_sold_select_statement
    ,employees_select_statement
    ,sales_select_statement
]

In [31]:

for table, statement in zip(table_names, select_statements_list):
    
    print('\n retrieving data from: ' + table + '\n')
    
    try:
        cursor.execute(statement)
        
        row = cursor.fetchone()
        
        while row:
            print(row)
            row = cursor.fetchone()        
        
    except psycopg2.Error as error:
        print("Error retrieving data")
        print(error)
    


 retrieving data from: transactions_3nf

(1, 'Amanda', 1, 2000)
(2, 'Toby', 1, 2000)
(3, 'Max', 2, 2018)

 retrieving data from: albums_sold

(1, 1, 'Rubber Soul')
(2, 1, 'Let it Be')
(3, 2, 'My Generation')
(4, 3, 'Meet the Beatles')
(5, 3, 'Help!')

 retrieving data from: employees

(1, 'Sam')
(2, 'Bob')

 retrieving data from: sales

(1, 40.0)
(2, 19.0)
(3, 45.0)


#### write a `SELECT` statement
that returns the columns below:  
`transaction_id`  
`customer_name`  
`cashier_name`  
`year`  
`albums_sold`  
`amount_sold`  

In [38]:

first_denormalized_select_statement = """
    SELECT
         transactions.transaction_id
        ,transactions.customer_name
        ,employees.employee_name AS cashier_name
        ,transactions.year
        ,albums_sold.album_name
        ,sales.amount_spent AS amount_sold
        
    FROM
        public.transactions_3nf AS transactions
        
    LEFT JOIN
        public.employees AS employees
        
    ON
        transactions.employee_id
        =
        employees.employee_id
        
    LEFT JOIN
        public.albums_sold AS albums_sold
        
    ON
        transactions.transaction_id
        =
        albums_sold.transaction_id
        
    LEFT JOIN
        public.sales AS sales
        
    ON
        transactions.transaction_id
        =
        sales.transaction_id
"""

In [40]:

try:
    cursor.execute(first_denormalized_select_statement)
    
    row = cursor.fetchone()
    
    while row:
        print(row)
        row = cursor.fetchone()
    
except psycopg2.Error as error:
    print("Error: couldn't retrieve data")
    print(error)

(1, 'Amanda', 'Sam', 2000, 'Rubber Soul', 40.0)
(1, 'Amanda', 'Sam', 2000, 'Let it Be', 40.0)
(2, 'Toby', 'Sam', 2000, 'My Generation', 19.0)
(3, 'Max', 'Bob', 2018, 'Meet the Beatles', 45.0)
(3, 'Max', 'Bob', 2018, 'Help!', 45.0)


#### add the `amount_spent` columns
to the `transactions_3nf` table

In [47]:

create_table_denormalized_transactions_statement = """
    DROP TABLE IF EXISTS
        public.denormalized_transactions
    ;
    
    CREATE TABLE
        public.denormalized_transactions(
             transaction_id SMALLINT
            ,customer_name TEXT
            ,employee_id SMALLINT
            ,year INTEGER
            ,amount_spent FLOAT8
        )
"""

denormalized_transactions_query = """
    TRUNCATE TABLE
        public.denormalized_transactions
    ;
    
    INSERT INTO
        public.denormalized_transactions(
             transaction_id
            ,customer_name
            ,employee_id
            ,year
            ,amount_spent
        )
        
    SELECT
         transactions.transaction_id
        ,transactions.customer_name
        ,transactions.employee_id
        ,transactions.year
        ,sales.amount_spent
        
    FROM
        public.transactions_3nf AS transactions
        
    LEFT JOIN
        public.sales AS sales
        
    ON
        transactions.transaction_id
        =
        sales.transaction_id
"""



#### create the `denormalized_transactions` table

In [49]:

try:
    cursor.execute(create_table_denormalized_transactions_statement)
    
except psycopg2.Error as error:
    print("Error: couldn't create table")
    print(error)

#### insert data
into the `denormalized_transactions` table

In [52]:

try:
    cursor.execute(denormalized_transactions_query)
   
except psycopg2.Error as error:
    print("Error: couldn't retrieve data")
    print(error)

#### run the first query
required by the exercise:  
`SELECT transaction_id, customer_name, amount_spent FROM ...`

In [53]:

first_exercise_query = """
    SELECT
         transaction_id
        ,customer_name
        ,amount_spent
        
    FROM
        public.denormalized_transactions
"""

In [55]:

try:
    cursor.execute(first_exercise_query)
    
    row = cursor.fetchone()
    
    while row:
        print(row)
        row = cursor.fetchone()
    
except psycopg2.Error as error:
    print("Error: couldn't retrieve data")
    print(error)

(1, 'Amanda', 40.0)
(2, 'Toby', 19.0)
(3, 'Max', 45.0)


#### the second exercise query is:
`SELECT cashier_name, SUM(amount_spent) FROM ... GROUP BY cashier_name`  
  
the `cashier_sales` table must be created containing the following columns:  
`transaction_id`  
`cashier_name`  
`cashier_id`  
`amount_spent` 


In [58]:

create_table_cashier_sales_statement = """
    DROP TABLE IF EXISTS
        public.cashier_sales
    ;
    
    CREATE TABLE
        public.cashier_sales(
             transaction_id SMALLINT
            ,cashier_name TEXT
            ,cashier_id SMALLINT
            ,amount_spent FLOAT8
        )
    ;
"""

insert_into_cashier_sales_table = """
    TRUNCATE TABLE
        public.cashier_sales
    ;
    
    INSERT INTO
        public.cashier_sales(
             transaction_id
            ,cashier_name
            ,cashier_id
            ,amount_spent
        )
    
    SELECT
         transactions.transaction_id
        ,employees.employee_name AS cashier_name
        ,transactions.employee_id AS cashier_id
        ,transactions.amount_spent
        
    FROM
        public.denormalized_transactions AS transactions
        
    INNER JOIN
        public.employees AS employees
        
    ON
        transactions.employee_id
        =
        employees.employee_id
"""

#### create the `cashier_sales` table

In [59]:

try:
    cursor.execute(create_table_cashier_sales_statement)
    
except psycopg2.Error as error:
    print("Error: couldn't create table")
    print(error)

#### then insert data
into `cashier_sales`

In [60]:

try:
    cursor.execute(insert_into_cashier_sales_table)
    
except psycopg2.Error as error:
    print("Error: couldn't insert data")
    print(error)

#### finally execute the required query:

In [63]:

second_exercise_query = """
    SELECT
         cashier_name
        ,SUM(amount_spent) AS amount_spent
        
    FROM
        public.cashier_sales
        
    GROUP BY
        cashier_name
"""

try:
    cursor.execute(second_exercise_query)
    
    row = cursor.fetchone()
    
    while row:
        print(row)
        
        row = cursor.fetchone()
        
except psycopg2.Error as error:
    print("Error: couldn't fetch data")
    print(error)

('Sam', 59.0)
('Bob', 45.0)


In [64]:

cursor.close()
connection.close()